# Программирование на Python 

*Алла Тамбовцева*

## Практикум 6. Введение в парсинг с `BeautifulSoup`

Вы уже знакомы со структурой HTML-страниц и основными тэгами, теперь попробуем выгрузить информацию с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам снова понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main` и отправим запрос к ней с помощью функции `get()` из `requests`:

In [2]:
main = "https://nplus1.ru/"
page = requests.get(main)

In [3]:
page

<Response [200]>

Заберём исходный код страницы и преобразуем строку с ним в объект `BeautifulSoup`:

In [4]:
soup = BeautifulSoup(page.text)

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [5]:
links_raw = soup.find_all("a") 
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/772">Красота</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/778">Научные закрытия</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/917">ИИ спешит на помощь</a>]

Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768', 
     'class' : 'hover:underline transition-colors duration-75'}.
    
Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [6]:
links = [li.get("href") for li in links_raw] 
links[10:20]  # несколько штук для примера

['/search/empty/768',
 '/search/empty/890',
 '/search/empty/871',
 '/search/empty/876',
 '/search/empty/775',
 '/search/empty/767',
 '/search/empty/771',
 '/search/empty/772',
 '/search/empty/778',
 '/search/empty/917']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [7]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li)

In [8]:
news[0:10]

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/05/19/octopus-nightmare',
 'https://nplus1.ru/news/2023/05/19/permafrost',
 'https://nplus1.ru/news/2023/05/19/ancient-hearth',
 'https://nplus1.ru/news/2023/05/19/x-ray-forgery-update',
 'https://nplus1.ru/news/2023/05/19/hoolock-tianxing',
 'https://nplus1.ru/news/2023/05/19/sn-2020-eyj',
 'https://nplus1.ru/news/2023/05/19/lp-791-18-d',
 'https://nplus1.ru/news/2023/05/19/octopus-nightmare',
 'https://nplus1.ru/news/2023/05/19/human-origins']

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [9]:
news = news[1:] 

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [10]:
link0 = news[0]
print(link0)

https://nplus1.ru/news/2023/05/19/octopus-nightmare


### Задача 1

Отправьте запрос к странице по ссылке `link0` с одной новостью, получите результат в виде объекта `BeautifulSoup` и сохраните его как `soup0`.

In [11]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

### Задача 2

Найдите заголовок новости и сохраните его в переменную `title`.

In [13]:
title = soup0.find("title").text

### Задача 3

Найдите имя автора новости и дату её публикации. Сохраните их в `author` и `date` соответственно.

In [14]:
date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
author = soup0.find("meta", {"name" : "author"}).get("content") 
print(date, author)

2023-05-19 Сергей Коленов


### Задача 4

Найдите сложность новости и рубрики, к которым она относится. Сохраните сложность в переменную `diffc`. Рубрики сначала можно сохранить в список, а затем его элементы объединить в одну строку `rubs`.

**Подсказка:** чтобы упростить себе жизнь, найдите сначала раздел (`div`), в котором на странице хранится дата и время публикации, сложность и рубрики, а затем выполняйте поиск в рамках этого раздела.

In [15]:
div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
diffc = div.find_all("span")[3].text
rubs_raw = div.find_all("span")[4:] 
rubs_L = [r.text for r in rubs_raw] 
rubs = ", ".join(rubs_L)
print(rubs)

Зоология


### Задача 5

Соберите из абзацев текст новости и сохраните его в переменную `text`. Избавьтесь от постронних символов (`\xa0`, `\n`) в тексте. 

In [16]:
pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
pars = [p.text for p in pars_raw] 
text = " ".join(pars)
text = text.replace("\xa0", " ").replace("\n", " ")
text

'Во сне он вел себя так, словно на него напал хищник Зоологи заподозрили, что головоногие моллюски могут видеть кошмары. Понаблюдав за живущим в неволе осьминогом по имени Костелло, исследователи выявили четыре случая, когда он во сне резко менял цвет, беспорядочно двигал телом и конечностями и даже выпускал чернила. Возможно, спящий моллюск видел сновидения с участием хищников — и пытался от них защититься или сбежать. Препринт исследования опубликован на сайте bioRxiv.  Сновидения приходят к людям во время фазы быстрого сна, когда активность мозга повышена, мышцы расслаблены, а глаза совершают быстрые движения. Для многих позвоночных, головоногих моллюсков и даже пауков также описана фаза быстрого сна или ее аналоги. Зоологи допускают, что во время нее эти животные видят сны. Тем не менее, понять, что им снится, трудно. Лишь недавно исследователи научились расшифровывать простейшие детали мышиных сновидений по движениям их глаз. Однако есть одна группа животных, эмоции которых отража

### Задача 6

Напишите функцию `get_news()`, которая принимает на вход ссылку на страницу с одной новостью, а возвращает список из следующих характеристик: имя автора, дата публикации, сложность новости, рубрики, текст новости.

*Да, мы пока не обсуждали написание функций в Python, но зато много раз сталкивались с ними в домашних заданиях.*

In [17]:
def get_news(link0):

    page0 = requests.get(link0)
    soup0 = BeautifulSoup(page0.text)
    title = soup0.find("title").text
    date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
    author = soup0.find("meta", {"name" : "author"}).get("content") 
    div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
    diffc = div.find_all("span")[3].text
    rubs_raw = div.find_all("span")[4:] 
    rubs_L = [r.text for r in rubs_raw] 
    rubs = ", ".join(rubs_L)
    pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
    pars = [p.text for p in pars_raw] 
    text = " ".join(pars)
    text = text.replace("\xa0", " ").replace("\n", " ")
    
    return title, author, date, diffc, rubs, text

Отлично! Теперь осталось применить функцию ко всем ссылкам в списке `news`. Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [18]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [19]:
# уменьшили задержку до 1 секунды в sleep

info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1)

https://nplus1.ru/news/2023/05/19/octopus-nightmare
https://nplus1.ru/news/2023/05/19/permafrost
https://nplus1.ru/news/2023/05/19/ancient-hearth
https://nplus1.ru/news/2023/05/19/x-ray-forgery-update
https://nplus1.ru/news/2023/05/19/hoolock-tianxing
https://nplus1.ru/news/2023/05/19/sn-2020-eyj
https://nplus1.ru/news/2023/05/19/lp-791-18-d
https://nplus1.ru/news/2023/05/19/octopus-nightmare
https://nplus1.ru/news/2023/05/19/human-origins
https://nplus1.ru/news/2023/05/19/dark-net-dark-bert
https://nplus1.ru/news/2023/05/18/gorillas-adverse
https://nplus1.ru/news/2023/05/18/rst-focal
https://nplus1.ru/news/2023/05/18/tonsillectomy-in-adults
https://nplus1.ru/news/2023/05/18/miliaria-crystallina
https://nplus1.ru/news/2023/05/18/optimus-tesla-bot-updates
https://nplus1.ru/news/2023/05/18/alisa-yandex-gpt
https://nplus1.ru/news/2023/05/18/primary-diffuse-meningeal-melanomatosis
https://nplus1.ru/news/2023/05/17/ufg-jwst
https://nplus1.ru/news/2023/05/17/siler-collingwoodi
https://nplus1

Посмотрим на несколько элементов `info`:

In [20]:
info[10:12]

[('Детские травмы не\xa0снизили продолжительность жизни взрослых горилл',
  'Катерина Петрова',
  '2023-05-18',
  '2.1',
  'Зоология',
  'Но увеличили риск смерти в раннем возрасте Гориллы, потерявшие в детстве родителя или пережившие рождение сиблинга, могут не дожить до шести лет. Но если доживут, то неблагоприятный детский опыт уже не сократит их жизнь, а в каких-то случаях даже увеличит ее продолжительность. Это выяснили исследователи из Великобритании, Руанды и США, которые уже более 50 лет наблюдают за гориллами в Национальном парке Вулканов. Исследование опубликовано в Current Biology. Негативный детский опыт обычно не очень хорошо влияет на жизнь уже взрослых людей: у них чаще возникают проблемы со здоровьем, а живут они хуже и меньше. Подобное происходит и со многими другими видами млекопитающих, особенно — с приматами [1, 2, 3]. Иногда последствия перенесенных в детстве травм могут догнать даже потомство. Но есть вид, у которого не совсем так, — это гориллы. Гориллы — наши до

Финальный штрих – импортируем `pandas` и преобразуемый полученный список кортежей в датафрейм:

In [21]:
import pandas as pd

In [22]:
df = pd.DataFrame(info)
df.head()

,0,1,2,3,4,5
0,Осьминогу приснились кошмары,Сергей Коленов,2023-05-19,1.7,Зоология,"Во сне он вел себя так, словно на него напал х..."
1,В Салехарде открыли первую скважину сети монит...,Илья Ферапонтов,2023-05-19,1.3,Экология и климат,Всего в сети будет 140 таких скважин Арктическ...
2,В Испании нашли очаги с сожженными примерно 24...,Михаил Подрезов,2023-05-19,3.3,"Археология, Химия","Это удалось выяснить, благодаря геохимическим ..."
3,Кембриджский центр кристаллографических данных...,Михаил Бойм,2023-05-19,1.1,Химия,Расследование подлога данных продолжается Кемб...
4,Хулоки Скайуокера проснулись пораньше ради фру...,Сергей Коленов,2023-05-19,1.7,Зоология,"Дело в том, что фруктовые деревья встречаются ..."


Добавим содержательные названия столбцов и выгрузим датафрейм в файл:

In [23]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]

Дополнительно: давайте еще изменим тип столбца `diffc`, сделаем его `float` вместо `object` (`object` – аналог `string` в `pandas`).

In [24]:
df["diffc"] = df["diffc"].astype(float)

In [25]:
df.to_excel("nplus1.xlsx")